# Install awscli:
```bash
$ pip install awscli

$ aws configure
AWS Access Key ID [None]: V42FCGRVMK24JJ8DHUYG
AWS Secret Access Key [None]: bKhWxVF3kQoLY9kFmt91l+tDrEoZjqnWXzY9Eza
Default region name [None]:
Default output format [None]:

$ cat ~/.aws/credentials
[default]
aws_access_key_id = V42FCGRVMK24JJ8DHUYG
aws_secret_access_key = bKhWxVF3kQoLY9kFmt91l+tDrEoZjqnWXzY9Eza
```

# How to use `boto3`
Sample as to how to initialize s3 client to work with Minio API compatible - https://github.com/minio/minio
AWS CLI counterpart - https://docs.minio.io/docs/aws-cli-with-minio

```python
import boto3

s3 = boto3.resource('s3', 
  endpoint_url='http://MINIO_HOST_NAME:9000', 
  config=boto3.session.Config(signature_version='s3v4')
)
```

# Refs.
1. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html


In [3]:
import boto3
import pandas as pd
from io import StringIO

s3 = boto3.resource('s3', 
  endpoint_url='http://minio:9000', 
  config=boto3.session.Config(signature_version='s3v4')
)

In [4]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

NoCredentialsError: Unable to locate credentials

In [ ]:
bucket_name = 'test'
object_key = 'wordcount/output'

obj = s3.Object(bucket_name, object_key)
csv = obj.get()['Body'].read().decode('utf-8')


In [ ]:
df = pd.read_csv(StringIO(csv), header=None, delimiter=' ')
df.columns = ['word', 'count']
df["count"] = pd.to_numeric(df["count"])
df.dtypes

sorted = df.sort_values(by=['count'], ascending=False)

sorted.head(10)

In [ ]:
import chartify

ch = chartify.Chart(blank_labels=True, x_axis_type='categorical')
    ch.set_title("Vertical bar plot")
    ch.set_subtitle("Automatically sorts by value counts.")
    ch.plot.bar(
        data_frame=sorted,
        categorical_columns='word',
        numeric_column='count')
    ch.show('png')